In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import glob
import cv2
import numpy as np
import skimage
images = [cv2.cvtColor(cv2.imread(file),cv2.COLOR_BGR2RGB) for file in glob.glob("../input/blood-cell-detection-dataset/images/*.png")]
images = np.array(images)
import matplotlib.pyplot as plt
plt.imshow(images[3,:,:,:])
plt.show()
image = images[3,:,:,:]

In [ ]:
norm_img = image.copy()
for i in range(image.shape[0]):
    for j in range(image.shape[1]):
        for k in range(image.shape[2]):
            norm_img[i,j,k] = (image[i,j,k]/np.sum(image[i,j,:]))*255
plt.imshow(norm_img)

In [ ]:
#-----Converting image to LAB Color model----------------------------------- 
lab = cv2.cvtColor(norm_img.copy(), cv2.COLOR_RGB2LAB)
plt.imshow(lab)
plt.show()
#-----Splitting the LAB image to different channels-------------------------
l, a, b = cv2.split(lab.copy())
plt.imshow(l ,'gray')
plt.show()
plt.imshow(a ,'gray')
plt.show()
plt.imshow(b ,'gray')
plt.show()
#-----Applying CLAHE to L-channel-------------------------------------------
clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
cl = clahe.apply(l.copy())
plt.imshow(cl)
plt.show()
#-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
limg = cv2.merge((cl.copy(),a.copy(),b.copy()))
plt.imshow(limg.copy())
plt.show()
#-----Converting image from LAB Color model to RGB model--------------------
final = cv2.cvtColor(limg.copy(), cv2.COLOR_LAB2RGB)
plt.imshow(final)
plt.show()
#_____END_____#

In [ ]:
invert_contrast_img = cv2.bitwise_not(final.copy())
plt.imshow(invert_contrast_img)
plt.show()

In [ ]:
a = ['RED','GREEN','BLUE']
for i in range(3):
    print(a[i])
    plt.imshow(invert_contrast_img[:,:,i],'gray')
    plt.axis('off')
    plt.show()
red_channel_img = invert_contrast_img[:,:,1]

In [ ]:
plt.imshow(red_channel_img,'gray')

In [ ]:
histogram_data = np.array(np.unique(red_channel_img, return_counts=True))
indices = list(histogram_data[0])
values = list(histogram_data[1])
bins = range(256)
hist_values = []
for i in range(256):
    if i in indices:
        hist_values.append(values[indices.index(i)])
    else:
        hist_values.append(0)
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(bins,hist_values)
plt.title('RED CHANNEL COLOR HISTOGRAM')
plt.show()

In [ ]:
avg_px = np.sum(np.multiply(indices, values))//np.sum(values)
print('avg = ',avg_px)
T = np.sum(np.multiply(indices[indices.index(avg_px):],values[indices.index(avg_px):]))//np.sum(values[indices.index(avg_px):])
print('T = ',T)

In [ ]:
frgnd_enhanced = red_channel_img.copy()
plt.imshow(frgnd_enhanced, 'gray')
plt.show()
for i in range(frgnd_enhanced.shape[0]):
    for j in range(frgnd_enhanced.shape[1]):
        if frgnd_enhanced[i,j]<=T:
            frgnd_enhanced[i,j] //= red_channel_img[i,j]
        else:
            frgnd_enhanced[i,j] = 0
plt.imshow(frgnd_enhanced, 'gray')
plt.show()

In [ ]:
gray_img = frgnd_enhanced.copy()
plt.imshow(gray_img,'gray')

blur_img = cv2.medianBlur(gray_img.copy(),5)

ret,th1 = cv2.threshold(blur_img,127,255,cv2.THRESH_BINARY + cv2.THRESH_OTSU)
th2 = cv2.adaptiveThreshold(blur_img,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
            cv2.THRESH_BINARY,11,2)
th3 = cv2.adaptiveThreshold(blur_img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,11,2)

titles = ['Original Image', 'Global Thresholding (v = 127)',
            'Adaptive Mean Thresholding', 'Adaptive Gaussian Thresholding']
images = [blur_img, th1, th2, th3]

for i in range(4):
    plt.subplot(2,2,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i])
    plt.xticks([]),plt.yticks([])
plt.show()

In [ ]:
invert_binary_img = cv2.bitwise_not(th1.copy())
plt.imshow(invert_binary_img, 'gray')

In [ ]:
indices[1:]

In [ ]:
import pandas as pd
annots = pd.read_csv('../input/blood-cell-detection-dataset/annotations.csv')

In [ ]:
def augment_colors(im):
    for i in range(im.shape[0]):
        for j in range(im.shape[1]):
            if(im[i,j,1]!=np.max(im[i,j,:])):
                if im[i,j,0]>im[i,j,2]:
                    im[i,j,2]/=im[i,j,0]
                elif im[i,j,0]<im[i,j,2]:
                    im[i,j,0]/=im[i,j,2]
            else:
                im[i,j,:] = [255,255,255]
    
    return im

In [ ]:
aug_img = augment_colors(image.copy())
plt.imshow(aug_img)
plt.axis('off')
plt.show()

In [ ]:
gray_img = cv2.cvtColor(aug_img.copy(),cv2.COLOR_BGR2GRAY)
plt.imshow(gray_img,'gray')

blur_img = cv2.medianBlur(gray_img.copy(),5)

ret,th1 = cv2.threshold(blur_img,127,255,cv2.THRESH_BINARY)
th2 = cv2.adaptiveThreshold(blur_img,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
            cv2.THRESH_BINARY,11,2)
th3 = cv2.adaptiveThreshold(blur_img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
            cv2.THRESH_BINARY,11,2)

titles = ['Original Image', 'Global Thresholding (v = 127)',
            'Adaptive Mean Thresholding', 'Adaptive Gaussian Thresholding']
images = [blur_img, th1, th2, th3]

for i in range(4):
    plt.subplot(2,2,i+1),plt.imshow(images[i],'gray')
    plt.title(titles[i])
    plt.xticks([]),plt.yticks([])
plt.show()

In [ ]:
border_min_img = cv2.bitwise_not(max_img.copy())
plt.imshow(border_min_img, 'gray')
plt.axis('off')
plt.show()

In [ ]:
border_img_2 = cv2.bitwise_not(th2.copy())
plt.imshow(border_img_2, 'gray')
plt.axis('off')
plt.show()

In [ ]:
avg_img = border_img_2.copy()
max_img = border_img_2.copy()
min_img = border_img_2.copy()
for i in range(border_img_2.shape[0]):
    for j in range(border_img_2.shape[1]):
        avg_img[i,j] = (border_img_3[i,j]+border_img_2[i,j])//2
        max_img[i,j] = max(border_img_3[i,j],border_img_2[i,j])
        min_img[i,j] = min(border_img_3[i,j],border_img_2[i,j])
plt.imshow(avg_img, 'gray')
plt.show()
plt.imshow(max_img, 'gray')
plt.show()
plt.imshow(min_img, 'gray')
plt.show()

In [ ]:
gray = min_img.copy() # convert to grayscale
blur = cv2.blur(gray, (3, 3)) # blur the image
ret, thresh = cv2.threshold(blur, 50, 255, cv2.THRESH_BINARY)
im2, contours = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

plt.show()

In [ ]:
i = 1
while i < 9:
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(i,i))
    closing = cv2.morphologyEx(border_img.copy(), cv2.MORPH_CLOSE, kernel)
    plt.imshow(closing)
    plt.axis('off')
    plt.show()
    i+=1

In [ ]:
plt.imshow(np.subtract(cv2.cvtColor(image.copy(),cv2.COLOR_BGR2GRAY),th3.copy()), 'gray')

In [ ]:
img_w_border = image.copy()
for i in range(image.shape[0]):
    for j in range(image.shape[1]):
        if img_w_border[i,j,1] != max(img_w_border[i,j,:]) and :
            

In [ ]:
hsv_im = cv2.cvtColor(aug_img.copy(), cv2.COLOR_BGR2HSV)
h,s,v = cv2.split(hsv_im)
plt.imshow(h,'gray')
plt.show()
plt.imshow(s,'gray')
plt.show()
plt.imshow(v,'gray')
plt.show()
(threshx, binary_img) = cv2.threshold(gray_img, 127, 0, cv2.THRESH_BINARY)
plt.imshow(binary_img,'gray')
plt.show()

In [ ]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gradient = cv2.morphologyEx(gray,cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,5)))
plt.imshow(gradient)
plt.show()
plt.imshow(gray)
plt.show()

In [ ]:
for i in range(gradient.shape[0]):
    for j in range(gradient.shape[1]):
        if gradient[i,j,1] == np.max(gradient[i,j,:]):
            gradient[i,j,1]*=2
            gradient[i,j,0]/=2
            gradient[i,j,2]/=2
        if gradient[i,j,0] == np.max(gradient[i,j,:]):
            gradient[i,j,0]*=2
            gradient[i,j,1]/=2
            gradient[i,j,2]/=2

plt.imshow(gradient)
plt.imshow(np.subtract(gradient, image))

In [ ]:
import skimage.morphology
plt.imshow(skimage.morphology.diameter_closing(image, diameter_threshold=8, connectivity=2))

In [ ]:
for i in range(1,6):
    increase_contrast(image.copy(),i)

In [ ]:
def increase_contrast(image):
    img = image.copy()
    lab= cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    f, axarr = plt.subplots(3,6)
    axarr[0,0].imshow(img)
    axarr[0,1].imshow(lab)
    axarr[0,2].imshow(l)
    axarr[0,3].imshow(a)
    axarr[0,4].imshow(b)

In [ ]:
def trial_grads(im):
    for i in range(im.shape[0]):
        for j in range(im.shape[1]):
            if(im[i,j,1]!=np.max(im[i,j,:])):
                if im[i,j,0]>im[i,j,2]:
                    im[i,j,2]/=im[i,j,0]
                elif im[i,j,0]<im[i,j,2]:
                    im[i,j,0]/=im[i,j,2]
            else:
                im[i,j,:] = [0,0,0]
    return im

In [ ]:
def preprocess_im(image):
    im = image
    for i in range(im.shape[0]):
        for j in range(im.shape[1]):
            if(im[i,j,1]!=np.max(im[i,j,:])):
                if im[i,j,0]<im[i,j,2]:
                    im[i,j,:] = [255,255,255]
                elif im[i,j,0]>im[i,j,2]:
                    im[i,j,:] = [0,0,0]
            else:
                im[i,j,:] = [0,0,0]
    return im

In [ ]:
plt.imshow(trial_grads(image))

In [ ]:
plt.imshow(binars_img,'gray')

In [ ]:
erosion_copy = binars_img.copy()
dilation_copy = binars_img.copy()

In [ ]:
dilation_copy.shape

In [ ]:
diamond = np.array([[0,0,1,0,0],[0,1,1,1,0],[1,1,1,1,1],[0,1,1,1,0],[0,0,1,0,0]])
diamond.shape

In [ ]:
square= np.array([[0,1,0],[1,1,1],[0,1,0]])
plt.imshow(cv2.dilate(dilation_copy,np.ones((2,2),int), iterations = 1),'gray')
plt.show()
dilated_img = cv2.dilate(dilation_copy,np.ones((2,2),int), iterations = 1)

In [ ]:
hsv_im = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
h,s,v = cv2.split(hsv_im)
plt.imshow(s,'gray')
plt.show()
(threshx, binaryx) = cv2.threshold(s, 127, 0, cv2.THRESH_BINARY)
plt.imshow(binaryx,'gray')
plt.show()

In [ ]:
plt.imshow(s,'gray')
plt.show()
plt.imshow(np.subtract(preprocess_im(image),s),'gray')
plt.show()

In [ ]:
im = preprocess_im(image)
gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
(thresh, binary) = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
plt.imshow(binary, 'gray')
plt.show()
kernel = np.ones((5,5), np.uint8) 
img_dilation = cv2.dilate(binary, kernel, iterations=1) 
plt.imshow(img_dilation, 'gray')

In [ ]:
def preprocess_img(im):
    for i in range(im.shape[0]):
        for j in range(im.shape[1]):
            try:
                if np.mean(im[i-2:i,j-2:j+2,:])>np.mean(im[i,j-2:j+2,:]) and np.mean(im[i+1:i+3,j-2:j+2,:])>np.mean(im[i,j-2:j+2,:]):
                   im[i+1:i+3,j-2:j+2,:] = im[i+1:i+3,j-2:j+2,:]*int(np.mean(im[i,j-2:j+2,:]))
            except:
                continue
    plt.imshow(im,'gray')

In [ ]:
gray = cv2.cvtColor(gradient, cv2.COLOR_BGR2GRAY)
(thresh, binary) = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
plt.imshow(binary, 'gray')

In [ ]:
contours, hierarchy = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
# draw all contours
new_image = cv2.drawContours(gradient, contours, -1, (0, 255, 0), 1)

In [ ]:
plt.imshow(new_image,'gray')

In [ ]:

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
  
# Find Canny edges 
edged = cv2.Canny(gray, 30, 200) 
cv2.waitKey(0) 

In [ ]:
output = new_image.copy()
gray = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY) 
circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1.2, 100)
# ensure at least some circles were found
if circles is not None:
    # convert the (x, y) coordinates and radius of the circles to integers
    circles = np.round(circles[0, :]).astype("int")
    # loop over the (x, y) coordinates and radius of the circles
    for (x, y, r) in circles:
        # draw the circle in the output image, then draw a rectangle
        # corresponding to the center of the circle
        cv2.circle(output, (x, y), r, (0, 255, 0), 4)
        cv2.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1)
    # show the output image
    cv2.imshow(output, np.hstack([image, output]))
    cv2.waitKey(0)

In [ ]:
plt.imshow(output)

In [ ]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
ret, thresh1 = cv2.threshold(gray, 120, 255, cv2.THRESH_BINARY + 
                                            cv2.THRESH_OTSU)      
plt.imshow(edged,'gray')
plt.imshow(np.add(gray, edged),'gray')

In [ ]:

# Finding Contours 
# Use a copy of the image e.g. edged.copy() 
# since findContours alters the image 
contours, hierarchy = cv2.findContours(edged,  
    cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
  
cv2.imshow('Canny Edges After Contouring', edged) 
cv2.waitKey(0) 
  
print("Number of Contours found = " + str(len(contours))) 

In [ ]:

  
# Draw all contours 
# -1 signifies drawing all contours 
cv2.drawContours(image, contours, -1, (0, 255, 0), 3) 
  
cv2.imshow('Contours', image) 

In [ ]:
  
# Draw all contours 
# -1 signifies drawing all contours 
cv2.drawContours(image, contours, -1, (0, 255, 0), 3) 
  
cv2.imshow('Contours', image) 

In [ ]:
import cv2
grayimg = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

plt.imshow(grayimg,'gray')

In [ ]:
kernel = np.ones((2,1),int)
equ = cv2.morphologyEx(image, cv2.MORPH_GRADIENT, kernel)
#gradient = cv2.equalizeHist(equ)
plt.imshow(equ)

In [ ]:
plt.imshow(image[:,:,0])
plt.show()
plt.imshow(image[:,:,1])
plt.show()
plt.imshow(image[:,:,2])
plt.show()
plt.imshow(image)
plt.show()

In [ ]:
image = cv2.cvtColor(image, cv2.COLOR_GRAY2BINARY)

In [ ]:
import scipy.ndimage
kernel = np.array([[-1,0,-1],[-2,0,2],[-1,0,1]])
gradient = cv2.morphologyEx(image,cv2.MORPH_GRADIENT, kernel)
plt.imshow(gradient)
plt.show()
plt.imshow(image)
plt.show()

In [ ]:
lap_kernel = np.array([[0,-1,0],[-1,4,-1],[0,-1,0]])
lap_kernel.shape

In [ ]:
c,i = 0,0
prevfile = ''
d = Dict()
for file in annots['image']:
    im = cv2.imread('../input/blood-cell-detection-dataset/images/'+file)
    label = annots.loc[i,'label']
    if prevfile == file:
        c+=1
    else:
        c = 0
    [_,xmin,ymin, xmax, ymax, _] = annots.loc[annots['image']==file,:].loc[c]
    [xmin, ymin, xmax, ymax] = [int(x) for x in [xmin, ymin, xmax, ymax]]
    if label in d.keys():
        d[label].append(im[xmin:xmax, ymin:ymax])
    else:
        d[label] = im[xmin:xmax, ymin:ymax]
    prevfile = file
    i+=1

In [ ]:
[_,xmin,ymin, xmax, ymax, _] = annots.loc[annots['image']=='image-100.png',:].loc[0]